In [5]:
def Battery_Failure_Risk_Calc(BatteryStatus, time, Lambda = 0.001, alpha = 0.008, beta = 0.007, Battery_degradation_rate = 0.0064):

    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    # Program Name : Markov-based Drone's Reliability and MTTF Estimator      %
    # Author       : Koorosh Aslansefat                                       %
    # Version      : 1.0.1                                                    %
    # Description  : A Markov Process-Based Approach for Reliability          %
    #                Evaluation of the Battery System for Drones              %
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

    ## Provided Markov model is from the following paper:
    # Kabir, S., Aslansefat, K., Sorokos, I., Papadopoulos, Y., & Gheraibia, Y. (2019, October). 
    # A conceptual framework to incorporate complex basic events in HiP-HOPS. 
    # In International Symposium on Model-Based Safety and Assessment (pp. 109-124). Springer, Cham.

    # Battery Status is an integer between 0 and 3. 0 means leass than 25% charged or fully decharged and 3 means  75% up to fully charged. 
    # Lambda: Failure Rate of the Battery System including Battery, voltage regulator and voltage/current meter.
    # alpha and beta: Are charge and discharge rates.

    ## Importing Required Libraries
    import numpy as np  # linear algebra
    import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
    import sympy as sym # Symbolic Calculation
    
    t = sym.Symbol('t')
    
    a = alpha
    b = beta
    d = Battery_degradation_rate
    L = Lambda
    
    if BatteryStatus == 3:
        P0_Battery = sym.Matrix([[1],[0],[0],[0],[0],[0],[0],[0]])
        Sflag = 5
    elif BatteryStatus == 2:
        P0_Battery = sym.Matrix([[0],[0],[1],[0],[0],[0],[0],[0]])
        Sflag = 3
    elif BatteryStatus == 1:
        P0_Battery = sym.Matrix([[0],[0],[0],[0],[1],[0],[0],[0]])
        Sflag = 1
    else:
        P_Fail = 1
        MTTF = 0

    M_Battery = sym.Matrix([[-L-a-d,  b,       0,  0,       0,  0, 0, 0],
                            [     a, -b,       0,  0,       0,  0, 0, 0],
                            [     d,  0,  -L-a-d,  b,       0,  0, 0, 0],
                            [     0,  0,       a, -b,       0,  0, 0, 0],
                            [     0,  0,       d,  0,  -L-a-d,  b, 0, 0],
                            [     0,  0,       0,  0,       a, -b, 0, 0],
                            [     0,  0,       0,  0,       d,  0, 0, 0],
                            [     L,  0,       L,  0,       L,  0, 0, 0]])  
    
    P_Battery = sym.exp(M_Battery*t)*P0_Battery
    
    P_Battery_Fail = P_Battery[-1] + P_Battery[-2]

    N_Battery = sym.Matrix([[-L-a-d,  b,       0,  0,       0,  0],
                            [     a, -b,       0,  0,       0,  0],
                            [     d,  0,  -L-a-d,  b,       0,  0],
                            [     0,  0,       a, -b,       0,  0],
                            [     0,  0,       d,  0,  -L-a-d,  b],
                            [     0,  0,       0,  0,       a, -b]])
 
    tt = -1*N_Battery.inv()
    MTTF = sum(tt[Sflag,:])

    return P_Battery_Fail.evalf(subs={t: time}), MTTF.evalf(subs={t: time})

In [6]:
P_Fail, MTTF = Battery_Failure_Risk_Calc(3, 100)

print(P_Fail)

print(MTTF)